* Use Case for Performance Testing
* Create a new API for orders
* Validate API for orders
* Performance Testing of the API
* Performance Testing of SQL
* Review Explain Plan and Fix the issue
* Retest the Performance
* Exercise and Solution - Performance Testing of order items

* Use Case for Performance Testing

Orders have 4 columns - order_id, order_date, order_customer_id and order_status. `order_id` is primary key in the table and hence it is indexed. We will do the performance testing to get the data based on order_id and order_customer_id.

* Create a new API for orders

1. Create a new model for orders table. Create a file by name `models/oms.py`. This file can have multiple models. Add the below code to `models/oms.py`.

```python
from app import db


class Order(db.Model):
    __tablename__ = 'orders'

    order_id = db.Column(db.Integer, primary_key=True)
    order_date = db.Column(db.Date)
    order_customer_id = db.Column(db.Integer)
    order_status = db.Column(db.String(50))

    def __repr__(self):
        return f"Order(order_id={self.order_id}, order_date={self.order_date}, " \
               f"order_customer_id={self.order_customer_id}, order_status={self.order_status})"
```

2. Add logic for order routes. Create a new file by name `routes/order_routes.py` using below code. It have the logic to get order details based on order id or order customer id.

```python
from flask import request, jsonify

from app import app
from models.order import Order, OrderItem


@app.route('/order')
def order():
    args = dict(request.args)
    order = None
    if 'order_id' in args.keys():
        order_id = args['order_id']
        order = Order.query.get(int(order_id))
        order.__dict__.pop('_sa_instance_state')
        return jsonify(order.__dict__)
    if 'order_customer_id' in args.keys():
        order_customer_id = args['order_customer_id']
        order_recs = Order.query.filter(Order.order_customer_id == order_customer_id).all()
        orders = []
        for order in order_recs:
            order.__dict__.pop('_sa_instance_state')
            orders.append(order.__dict__)
        return jsonify(orders)
```

3. Update `app.py` - add order_routes to the imports

* Validate API for orders

1. Run the application using `python -m flask run --debug`.
2. Use `requests` to validate.

In [ ]:
import requests

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
order_id = int(input('Enter order id: '))

In [ ]:
requests.get(f'{base_url}/order?order_id={order_id}').json()

In [ ]:
order_customer_id = int(input('Enter order customer id: '))

In [ ]:
requests.get(f'{base_url}/order?order_customer_id={order_customer_id}').json()

* Performance Testing of the API

1. Randomize 10000 order ids and test `/order?order_id=<order_id>` end point.
2. Randomize 10000 order customer ids and test `/order?order_customer_id=<order_customer_id>` end point.

In [ ]:
import requests
import random

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
%%time
for i in range(1, 10000):
    if i % 1000 == 0:
        print('.', end='')
    p = random.randint(1, 10000)
    payload = requests.get(f'{base_url}/order?order_id={p}').json()

In [ ]:
%%time
for i in range(1, 10000):
    if i % 1000 == 0:
        print('.', end='')
    p = random.randint(1, 10000)
    payload = requests.get(f'{base_url}/order?order_customer_id={p}').json()

* Performance Testing of SQL

1. Use `psycopg2` to run SQL queries directly.
2. Randomize 10000 order ids and run `SELECT * FROM orders WHERE order_id = %s`.
3. Randomize 10000 order customer ids and run `SELECT * FROM orders WHERE order_customer_id = %s`

Note: Running direct SQL queries yield us better performance as there is no overhead to interact with web application.

In [ ]:
import psycopg2

In [ ]:
conn = psycopg2.connect(
    host='localhost',
    port=5432,
    database='sales_db',
    user='sales_user',
    password='itversity'
)

In [ ]:
cur = conn.cursor()

In [ ]:
query = 'SELECT * FROM orders WHERE order_id = %s'

In [ ]:
cur.execute(query, (1, ))

In [ ]:
cur.fetchone()

In [ ]:
import random

In [ ]:
recs = []
for i in range(1, 10001):
    query = 'SELECT * FROM orders WHERE order_id = %s'
    p = random.randint(1, 60000)
    cur.execute(query, (p, ))
    recs.append(cur.fetchone())

In [ ]:
len(recs)

In [ ]:
recs[0]

In [ ]:
recs = []
for i in range(1, 10001):
    query = 'SELECT * FROM orders WHERE order_customer_id = %s'
    p = random.randint(1, 10000)
    cur.execute(query, (p, ))
    recs.append(cur.fetchall())

In [ ]:
len(recs)

In [ ]:
recs[0]

* Review Explain Plan and Fix the issue

1. Enable query logging and launch `flask shell`
2. Invoke below APIs and capture sql.

```python
from werkzeug.test import Client
client = Client(app)

client.get('/order?order_id=1').get_json()
client.get('/order?order_customer_id=1').get_json()
```

3. Go to pgAdmin and run `EXPLAIN` command to generate explain plan.
4. You will observe that the query related to `order_id` uses index scan where as the other query related to `order_customer_id` uses sequence or full table scan.
5. Create index on `order_customer_id`.

```sql
CREATE INDEX orders_order_customer_id_idx
ON orders(order_customer_id);
```

* Retest the Performance using SQL

It is enough to retest the performance using SQL. The performance based on `order_id` and `order_customer_id` will be almost same. Optionally, you can also run performance test using `requests`.

* Exercise - Performance Testing of order items

We will evaluate the performance of end points or routes related to order item and fix if there is any performance bottleneck.

1. Create end point for `order_item_id` (primary key) and `order_item_order_id`. You can use `orm.py` to add the model by name `OrderItem` based on the fields in order_items table. You can consider using Chat GPT.
2. Use SQL approach and evaluate performance for 10,000 randomly generated order item ids and order item order ids.
3. Generate explain plans, review them and identify any performance bottlenecks.
4. Fix the issue and evaluate the performance again.

* Solution - Performance Testing of order items

We will evaluate the performance of end points or routes related to order item and fix if there is any performance bottleneck.

1. Create end point for `order_item_id` (primary key) and `order_item_order_id`. You can use `orm.py` to add the model by name `OrderItem` based on the fields in order_items table. You can consider using Chat GPT.

Update `models/orm.py` for `OrderItem`.

```python
class OrderItem(db.Model):
    __tablename__ = 'order_items'

    order_item_id = db.Column(db.Integer, primary_key=True)
    order_item_order_id = db.Column(db.Integer)
    order_item_product_id = db.Column(db.Integer)
    order_item_quantity = db.Column(db.Integer)
    order_item_subtotal = db.Column(db.Float)
    order_item_product_price = db.Column(db.Float)

    def __repr__(self):
        return f"OrderItem(order_item_id={self.order_item_id}, " \
               f"order_item_order_id={self.order_item_order_id}, " \
               f"order_item_product_id={self.order_item_product_id}, " \
               f"order_item_quantity={self.order_item_quantity}, " \
               f"order_item_subtotal={self.order_item_subtotal}, " \
               f"order_item_product_price={self.order_item_product_price})"
```

Update `routes/order_routes.py` for `/order_item` end point or route. Make sure to import `OrderItem` from `models.oms` along with `Order`.

```python
@app.route('/order_item')
def order_item():
    args = dict(request.args)
    order_item = None
    if 'order_item_id' in args.keys():
        order_item_id = args['order_item_id']
        order_item = OrderItem.query.get(int(order_item_id))
        order_item.__dict__.pop('_sa_instance_state')
        return jsonify(order_item.__dict__)
    if 'order_item_order_id' in args.keys():
        order_item_order_id = args['order_item_order_id']
        order_item_recs = OrderItem.query.filter(OrderItem.order_item_order_id == order_item_order_id).all()
        order_items = []
        for order_item in order_item_recs:
            order_item.__dict__.pop('_sa_instance_state')
            order_items.append(order_item.__dict__)
        return jsonify(order_items)
```

2. Use SQL approach and evaluate performance for 10,000 randomly generated order item ids and order item order ids.

```python
import random
recs = []
for i in range(1, 10001):
    query = 'SELECT * FROM order_items WHERE order_item_id = %s'
    p = random.randint(1, 60000)
    cur.execute(query, (p, ))
    recs.append(cur.fetchone())

recs = []
for i in range(1, 10001):
    query = 'SELECT * FROM order_items WHERE order_item_order_id = %s'
    p = random.randint(1, 60000)
    cur.execute(query, (p, ))
    recs.append(cur.fetchone())
```

3. Generate explain plans, review them and identify any performance bottlenecks.

```sql
EXPLAIN SELECT * FROM order_items WHERE order_item_id = 1;

EXPLAIN SELECT * FROM order_items WHERE order_item_order_id = 2;
```

4. Fix the issue and evaluate the performance again.

```sql
CREATE INDEX order_items_order_item_order_id_idx
ON order_items (order_item_order_id);
```

```python
import random
recs = []
for i in range(1, 10001):
    query = 'SELECT * FROM order_items WHERE order_item_id = %s'
    p = random.randint(1, 60000)
    cur.execute(query, (p, ))
    recs.append(cur.fetchone())

recs = []
for i in range(1, 10001):
    query = 'SELECT * FROM order_items WHERE order_item_order_id = %s'
    p = random.randint(1, 60000)
    cur.execute(query, (p, ))
    recs.append(cur.fetchone())
```